In [1]:
# Resources

# Llama3.1 fine tuning example
# https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/finetuning/quickstart_peft_finetuning.ipynb


In [2]:
!pip install -q accelerate 
!pip install -q peft 
!pip install -q bitsandbytes 
!pip install -q trl

# The following packages are already installed on Kaggle or
# we won't be using them.

!pip install -q pandas
!pip install -q transformers 
#!pip install -q torch
!pip install -q datasets 
!pip install -q flash_attn  
#!pip install -q wandb


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import os

import transformers

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    set_seed
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from datasets import load_dataset

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

print(f"CUDA Version {torch.version.cuda}")
print(f"Pytorch {torch.__version__}")
print(f"Transformers {transformers.__version__}")

CUDA Version 12.1
Pytorch 2.1.1+cu121
Transformers 4.45.0


In [4]:
#from kaggle_secrets import UserSecretsClient

#user_secrets = UserSecretsClient()
#HF_ACCESS_TOKEN = user_secrets.get_secret("HF_ACCESS_TOKEN")

In [5]:
# Check the type and quantity of GPUs
# Note: Kaggle GPUs don't support flash attention

if torch.cuda.is_available():
    print('Num CPUs:', os.cpu_count())
    print('Num GPUs:', torch.cuda.device_count())
    print('GPU Type:', torch.cuda.get_device_name(0))
    
    # Check if bf16 is suported
    print("bf16 supported:",torch.cuda.is_bf16_supported())

Num CPUs: 96
Num GPUs: 1
GPU Type: NVIDIA A40
bf16 supported: True


## Settings

In [6]:
# Check if the model can overfit one batch
TRAIN_ONE_BATCH = False

MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"

MODEL_SAVE_PATH = "Phi-3.5-mini-instruct-ftuned"

DATA_PATH = 'fine_tune_data_phi3-5.csv'

NUM_VAL_SAMPLES = 100

DEVICE = "cuda"

In [7]:
import os

#os.listdir('../input/exp49-youth-phi3-mini-prep-fin-tun-data/')
 

## Prompt template

In [8]:
system_message = "You are a helpful assistant."
prompt = "Hello"
response = "Hello! I am a helpful assistant."

## Llama3.1
#print("Llama3.1 prompt template:")
#print(f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{response}<|eot_id|>")

## Phi3.5
text = f"<|system|>{system_message}<|end|><|user|>{prompt}<|end|><|assistant|>{response}<|end|>"
print(text)

<|system|>You are a helpful assistant.<|end|><|user|>Hello<|end|><|assistant|>Hello! I am a helpful assistant.<|end|>


## Prepare the data

The dataset that we will use for fine tuning will have one column named "text". Each row will be formatted using the prompt template shown above. The dataset that we pass to the model must be a huggingface dataset, not a pandas dataframe.

## Load the data

In [9]:
path = DATA_PATH
df_data = pd.read_csv(path)

print(df_data.shape)

df_data.head(1)

(3900, 32)


,uid,NarrativeLE,NarrativeCME,DepressedMood,MentalIllnessTreatmentCurrnt,HistoryMentalIllnessTreatmnt,SuicideAttemptHistory,SuicideThoughtHistory,SubstanceAbuseProblem,MentalHealthProblem,...,DisclosedToOtherFamilyMember,DisclosedToFriend,InjuryLocationType,WeaponType1,fold,json_labels,text,text_no_sys,num_tokens,num_tokens_no_sys
0,aaaf,V (XX XX) shot himself in a motor vehicle.The ...,V (XX XX) shot himself in a motor vehicle.The ...,0,0,0,0,1,0,0,...,1,0,2,5,3.0,"{""DepressedMood"": 0, ""MentalIllnessTreatmentCu...",<|system|>\nYour are an expert legal assistant...,<|user|>Law enforcement report: V (XX XX) shot...,1936,425


In [10]:
# Keep fold 0 aside for validation

df_train = df_data[df_data['fold'] != 0]
df_train = df_train.reset_index(drop=True)

df = df_data[df_data['fold'] == 0]

# Choose 100 samples
df_val = df[0:NUM_VAL_SAMPLES]

# Include the fold0 data
df2 = df[NUM_VAL_SAMPLES:]
df_train = pd.concat([df_train, df2], axis=0)
df_train = df_train.reset_index(drop=True)


df_val = df_val.reset_index(drop=True)


cols = ['uid', 'text', 'num_tokens']
df_train = df_train[cols]
df_val = df_val[cols]

print(df_train.shape)
print(df_val.shape)

(3800, 3)
(100, 3)


In [11]:
df_train.head(2)

,uid,text,num_tokens
0,aaaf,<|system|>\nYour are an expert legal assistant...,1936
1,aacl,<|system|>\nYour are an expert legal assistant...,2196


In [12]:
df_val.head(2)

,uid,text,num_tokens
0,aaby,<|system|>\nYour are an expert legal assistant...,1974
1,aacn,<|system|>\nYour are an expert legal assistant...,2718


In [13]:
# Check num tokens
print(df_train['num_tokens'].max())
print(df_train['num_tokens'].mean())
print(df_train['num_tokens'].min())

4294
2228.869210526316
1875


In [14]:
# Check num tokens
print(df_val['num_tokens'].max())
print(df_val['num_tokens'].mean())
print(df_val['num_tokens'].min())

3369
2236.97
1928


In [15]:
# Filter out rows that have > 4096 tokens

#df_data = df_data[df_data['num_tokens'] <= 4096]
#df_data = df_data.reset_index(drop=True)

#print(df_data.shape)

#df_data.head()

## Inference example

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    #use_auth_token=HF_ACCESS_TOKEN
    attn_implementation="flash_attention_2",
    use_cache=False,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                         #use_auth_token=HF_ACCESS_TOKEN
                                         )


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [17]:
def run_llm(system_message, user_message):

    #prompt = f"<|system|>{system_message}<|end|><|user|>{user_message}<|end|><|assistant|>"
    
    # Llama3
    #prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    
    # Phi3.5
    prompt = f"<|system|>{system_message}<|end|><|user|>{user_message}<|end|><|assistant|>"
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Send the inputs to the device
    inputs = inputs.to(DEVICE)

    # Generate the outputs from prompt
    generate_ids = model.generate(**inputs, 
                                  max_new_tokens=512,
                                  temperature=0.1,
                                  )


    # Decode the generated output
    generated_text = tokenizer.batch_decode(generate_ids, 
                                        skip_special_tokens=False,
                                        clean_up_tokenization_spaces=False)[0]
    
    print(generated_text)


system_message = "You are a helpful assistant."
user_message = "Hello"

run_llm(system_message, user_message)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


<|system|> You are a helpful assistant.<|end|><|user|> Hello<|end|><|assistant|> Hello! I'm Phi, an AI digital assistant. How can I help you today?<|end|>


## Delete the model and tokenizer

In [18]:
import gc

del model
del tokenizer

torch.cuda.empty_cache()
gc.collect()
gc.collect()

0

## Create single batch training set

We will use this to see it the model can be fine tuned to overfit one batch.

In [19]:
if TRAIN_ONE_BATCH == True:
    
    # Choose the first 4 rows
    df_train = df_train[0:4]

    print(df_train.shape)
    

## Convert to Huggingface dataset

In [20]:
from datasets import Dataset

# Convert the pandas DataFrame to a Hugging Face Dataset
hf_dataset_train = Dataset.from_pandas(df_train[['text']])
hf_dataset_val = Dataset.from_pandas(df_val[['text']])


In [21]:
hf_dataset_train

Dataset({
    features: ['text'],
    num_rows: 3800
})

In [22]:
hf_dataset_val

Dataset({
    features: ['text'],
    num_rows: 100
})

In [23]:
#hf_dataset_train[1]

## Set up parameters

In [24]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, 
    device_map="auto",
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attenstion support
    torch_dtype=torch.bfloat16,
)


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, 
                                          trust_remote_code=True)

#tokenizer.model_max_length = 2048
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'



peft_config = {
    "r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": "CAUSAL_LM",
    "target_modules": "all-linear",
    "modules_to_save": None,
}

peft_conf = LoraConfig(**peft_config)


training_config = {
    "bf16": True,
    "do_eval": False, # False
    "learning_rate": 5.0e-06,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 2,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    }

train_conf = TrainingArguments(**training_config)


trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=hf_dataset_train,
    eval_dataset=hf_dataset_val,
    max_seq_length=4300,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=False # True
)


"""
How to set up dynamic padding:
The dataset needs to be tokenized. There needs to be columns named: "input_ids", "attention_mask"

DataCollatorWithPadding: Automatically pads the sequences in each batch to the 
length of the longest sequence in that batch. This makes training more efficient, 
as padding is done dynamically per batch.
padding=True: When you set padding=True, it pads each sequence to the maximum length 
in the current batch, not globally across the dataset.



# Data collator:
#---------------

from transformers import DataCollatorWithPadding

# Use the tokenizer you've already loaded
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=hf_dataset_train,
    eval_dataset=hf_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,  # Include this line
    packing=False  # Ensure packing is turned off for individual sequences
)


# How to create a custom data collator:
#----------------------------------------

from torch.nn.utils.rnn import pad_sequence
import torch

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, features):
        # Extract input_ids and attention_mask
        input_ids = [torch.tensor(f["input_ids"]) for f in features]
        attention_mask = [torch.tensor(f["attention_mask"]) for f in features]

        # Pad the sequences dynamically based on the longest sequence in the batch
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)

        # Return a dictionary as expected by the model
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        }

data_collator = CustomDataCollator(tokenizer=tokenizer)

trainer = SFTTrainer(
    model=model,
    args=train_conf,
    peft_config=peft_conf,
    train_dataset=hf_dataset_train,
    eval_dataset=hf_dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,  # Custom collator
    packing=False
)


# More info:
# -----------

# Assuming your dataset has a 'text' field with the input text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True)

# Tokenize the dataset
hf_dataset_train = hf_dataset_train.map(tokenize_function, batched=True)
hf_dataset_val = hf_dataset_val.map(tokenize_function, batched=True)

# This will add 'input_ids' and 'attention_mask' fields to your dataset


"""


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/3800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend


In [25]:
# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

train_result = trainer.train()

metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** Running training *****
  Num examples = 3,800
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1,900
  Number of trainable parameters = 25,165,824


Step,Training Loss
20,1.363500
40,1.358900
60,1.375400
80,1.368600
100,1.342800
120,1.329400
140,1.316900
160,1.298500
180,1.272400
200,1.246600


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3.5-mini-instruct/snapshots/af0dfb8029e8a74545d0736d30cb6b58d2f0f3f0/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3.5-mini-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3.5-mini-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3.5-mini-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 3200

***** train metrics *****
  epoch                    =         2.0
  total_flos               = 397365763GF
  train_loss               =      0.5194
  train_runtime            =  2:31:50.37
  train_samples_per_second =       0.834
  train_steps_per_second   =       0.209


In [ ]:

#############
# Evaluation
#############
tokenizer.padding_side = 'right'
metrics = trainer.evaluate()
metrics["eval_samples"] = len(hf_dataset_val)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)


# ############
# # Save model
# ############
trainer.save_model(MODEL_SAVE_PATH)


***** Running Evaluation *****
  Num examples = 100
  Batch size = 4


In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()
gc.collect()

In [ ]:
# Create a requirements.txt file
!pip freeze > requirements.txt